In [1]:
# Import python utils
import torch
import math
from torch.utils import data
from torch import optim
from torch import Tensor
from torch import nn
from torch.nn import functional as F
from torchvision import datasets

In [2]:
# Import project utils
from utils.dlc_practical_prologue import generate_pair_sets
from utils.metrics import accuracy
from utils.nn_utils import train_model

# Import our models. 
from models.ResNet import residual_net
from models.LinearReluNet import linear_relu_net
from models.SimpleConvNet import simple_conv_net
from models.LeNet import lenet

In [3]:
# Data related global variables
N = 1000  # Number of pairs

# Neural nets parameters
NN_ARGS = {"input_size": 14 * 14, "nb_classes": 10}
BATCH_SIZE = 100
NUM_EPOCHS = 25

In [4]:
def build_data(number_of_pairs, batch_size):
    train_input, train_target, train_classes, test_input, test_target, test_classes = generate_pair_sets(number_of_pairs)
    train_dataset = data.TensorDataset(train_input, train_classes)
    test_dataset = data.TensorDataset(test_input, test_target)
    
    train_loader = data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader  
    

In [5]:
# Create the model & the data loader.
train_loader, test_loader = build_data(N, BATCH_SIZE)
#model = linear_relu_net(**NN_ARGS)
model = simple_conv_net()
model

SimpleConvNet(
  (cnn_layers): Sequential(
    (C1): Conv2d(1, 6, kernel_size=(2, 2), stride=(1, 1))
    (BN1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Relu1): ReLU()
    (S2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (C3): Conv2d(6, 7, kernel_size=(2, 2), stride=(1, 1))
    (BN2): BatchNorm2d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Relu3): ReLU()
    (S4): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (C5): Conv2d(7, 100, kernel_size=(2, 2), stride=(1, 1))
    (BN3): BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Relu5): ReLU()
  )
  (linear_layers): Sequential(
    (F6): Linear(in_features=100, out_features=84, bias=True)
    (Relu6): ReLU()
    (F7): Linear(in_features=84, out_features=10, bias=True)
    (LogSoftmax): LogSoftmax()
  )
)

In [6]:
model2 = lenet()
model2

LeNet(
  (conv_net): Sequential(
    (C1): Conv2d(1, 6, kernel_size=(2, 2), stride=(1, 1))
    (Relu1): ReLU()
    (S2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (C3): Conv2d(6, 7, kernel_size=(2, 2), stride=(1, 1))
    (Relu3): ReLU()
    (S4): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (C5): Conv2d(7, 120, kernel_size=(2, 2), stride=(1, 1))
    (Relu5): ReLU()
  )
  (fully_connected): Sequential(
    (F6): Linear(in_features=120, out_features=84, bias=True)
    (Relu6): ReLU()
    (F7): Linear(in_features=84, out_features=10, bias=True)
    (LogSoftmax): LogSoftmax()
  )
)

In [7]:
# Train model and evaluate performances
train_model(model, NUM_EPOCHS, train_loader, flatten=False)
print(accuracy(model2, test_loader, flatten=False))

End of Epoch 1, Avg Loss=4.1764
End of Epoch 2, Avg Loss=3.1153
End of Epoch 3, Avg Loss=2.1299
End of Epoch 4, Avg Loss=1.5128
End of Epoch 5, Avg Loss=1.1789
End of Epoch 6, Avg Loss=0.9832
End of Epoch 7, Avg Loss=0.8235
End of Epoch 8, Avg Loss=0.7486
End of Epoch 9, Avg Loss=0.6753
End of Epoch 10, Avg Loss=0.6172
End of Epoch 11, Avg Loss=0.5681
End of Epoch 12, Avg Loss=0.5311
End of Epoch 13, Avg Loss=0.5037
End of Epoch 14, Avg Loss=0.4437
End of Epoch 15, Avg Loss=0.4264
End of Epoch 16, Avg Loss=0.4026
End of Epoch 17, Avg Loss=0.3702
End of Epoch 18, Avg Loss=0.3625
End of Epoch 19, Avg Loss=0.3236
End of Epoch 20, Avg Loss=0.3162
End of Epoch 21, Avg Loss=0.2850
End of Epoch 22, Avg Loss=0.2675
End of Epoch 23, Avg Loss=0.2544
End of Epoch 24, Avg Loss=0.2378
End of Epoch 25, Avg Loss=0.2201
0.46


In [8]:
shallow_model = nn.Sequential(
    nn.Linear()
)

TypeError: __init__() missing 2 required positional arguments: 'in_features' and 'out_features'

In [ ]:
resnet =